In [1]:
import os
os.environ["HUGGINGFACE_HUB_CACHE"] = "/data/sshadmin/bocheng/.cache/huggingface/hub/"

In [2]:
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionImg2ImgPipeline,
    StableDiffusionInpaintPipeline,
    StableDiffusionDepth2ImgPipeline,
)
import torch
device="cuda"

# 载入管线
model_id = "stabilityai/stable-diffusion-2-1-base"
# pipe = StableDiffusionPipeline.from_pretrained(model_id).to(device)
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, revision="fp16", torch_dtype=torch.float16
).to(device)

# 查看pipe的组件
print(list(pipe.components.keys()))

/data/sshadmin/bocheng/soft/installed/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00,  7.85it/s]
/data/sshadmin/bocheng/soft/installed/miniconda3/lib/python3.11/site-packages/diffusers/pipelines/pipeline_utils.py:263: FutureWarning: You are loading the variant fp16 from stabilityai/stable-diffusion-2-1-base via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(
text_encoder/model.safetensors not found
Loading pipeline components...: 100%|██████████| 5/5 [00:02<00:00,  1.71it/s]


['vae', 'text_encoder', 'tokenizer', 'unet', 'scheduler', 'safety_checker', 'feature_extractor', 'image_encoder']


In [ ]:
pipe.vae

In [10]:
# 创建取值区间为(-1, 1)的伪数据
images = torch.rand(1, 3, 512, 512, dtype=torch.float16).to(device) * 2 - 1
print("Input images shape:", images.shape)

# 编码到隐空间
with torch.no_grad():
    latents = 0.18215 * pipe.vae.encode(images).latent_dist.mean
    #
print("Encoded latents shape:", latents.shape)

Input images shape: torch.Size([1, 3, 512, 512])
Encoded latents shape: torch.Size([1, 4, 64, 64])


In [11]:
with torch.no_grad():
    decoded_images=pipe.vae.decode(latents / 0.18215).sample

print("Decoded images shape:",decoded_images.shape)

Decoded images shape: torch.Size([1, 3, 512, 512])


In [12]:
input_ids=pipe.tokenizer(["A painting of a flooble"])['input_ids']
print("Input ID-> decoded token")
for input_id in input_ids[0]:
    print(f"{input_id} -> {pipe.tokenizer.decode(input_id)}")

Input ID-> decoded token
49406 -> <|startoftext|>
320 -> a
3086 -> painting
539 -> of
320 -> a
4062 -> floo
1059 -> ble
49407 -> <|endoftext|>


In [13]:
input_ids=torch.tensor(input_ids).to(device)
with torch.no_grad():
    text_embedding=pipe.text_encoder(input_ids)['last_hidden_state']
print("Text embedding shape:",text_embedding.shape)

Text embedding shape: torch.Size([1, 8, 1024])


In [21]:
text_embeddings = pipe.encode_prompt("A painting of a flooble", device=device, num_images_per_prompt=1, do_classifier_free_guidance=False)
print(text_embeddings)

(tensor([[[-0.3135, -0.4475, -0.0083,  ...,  0.2544, -0.0327, -0.2959],
         [ 0.1989, -1.6934, -0.8955,  ...,  0.4661, -0.0961, -2.1465],
         [ 0.4114, -1.1123, -0.4707,  ...,  1.8330,  0.6245, -0.4375],
         ...,
         [ 0.5273, -1.6484, -0.0616,  ..., -0.2157, -0.0891,  0.0115],
         [ 0.5845, -1.6514, -0.1008,  ..., -0.4829, -0.1517,  0.1576],
         [ 0.3491, -2.3984, -0.8232,  ..., -0.2310, -0.0977,  0.8594]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<ViewBackward0>), None)


In [22]:
text_embeddings[0].shape

torch.Size([1, 77, 1024])

In [27]:
pipe.scheduler.set_timesteps(30,device=device)

In [30]:
pipe.scheduler.timesteps.astype

tensor([958, 925, 925, 892, 859, 826, 793, 760, 727, 694, 661, 628, 595, 562,
        529, 496, 463, 430, 397, 364, 331, 298, 265, 232, 199, 166, 133, 100,
         67,  34,   1], device='cuda:0')

In [31]:
pipe.scheduler.timesteps[0]

tensor(958, device='cuda:0')

In [34]:
timestep = pipe.scheduler.timesteps[0]
latents = torch.randn(1, 4, 64, 64, dtype=torch.float16).to(device)
text_embedding = torch.randn(1, 77, 1024, dtype=torch.float16).to(device)
with torch.no_grad():
    unet_output = pipe.unet(latents, timestep.type_as(latents), text_embeddings).sample


AttributeError: 'tuple' object has no attribute 'shape'